(normal)=
# Normal Distribution

```{admonition} Important Readings
:class: seealso
- {cite}`freedman2007statistics`, Chapter 6
```

## The Normal Curve

The normal curve can be used as an ideal histogram, as if we had an enormous collection of observations for a continuous quantitative variable. The normal curve is symmetric and bell-shaped (though not all symmetric and bell-shaped distributions necessarily follow the normal curve). 

```{figure} images/normalcurve.svg
:width: 90%
:name: normalcurve

The standard normal curve with an average of zero and a standard deviation of one.
```

## Finding Areas Under the Normal Curve



In [2]:
from bokeh.io import show, output_notebook, output_file
from bokeh.layouts import column
from bokeh.models import CustomJS, Slider, ColumnDataSource, Div
from bokeh.plotting import figure
from scipy.stats import norm
import numpy as np
#import ipywidgets as widgets
from IPython.display import display, HTML

#with widgets.Output():
output_notebook(hide_banner = True)

# Standard normal curve data
x = np.linspace(-6, 6, 1000)
y = norm.pdf(x)
source = ColumnDataSource(data=dict(x=x, y=y, y_shaded=y))

# Bokeh plot setup
plot = figure(title="Area under Standard Normal Curve", tools="save",
              x_range=[-7, 7], y_range=[0, max(y)*1.1], width = 480, height = 400)

plot.line('x', 'y', source=source, line_width=2)
shaded_area = plot.varea(x='x', y1=0, y2='y_shaded', source=source, alpha=0.3)

# Remove Y axis and grid
plot.yaxis.visible = False
plot.ygrid.visible = False
plot.xgrid.visible = False

# Area text
area_text = Div(text="Area: 100.00%")

# JavaScript callback
callback = CustomJS(args=dict(source=source, area_text=area_text), code="""
    const data = source.data;
    const left_limit = left_slider.value;
    const right_limit = right_slider.value;
    const x = data['x'];
    const y = data['y'];
    const y_shaded = data['y_shaded'];

    let area = 0;
    for (let i = 0; i < x.length; i++) {
        if (x[i] >= left_limit && x[i] <= right_limit) {
            y_shaded[i] = y[i];
            area += y[i] * (x[1] - x[0]); // Approximate area calculation
        } else {
            y_shaded[i] = 0;
        }
    }

    const total_area = norm_cdf(right_limit) - norm_cdf(left_limit);
    area_text.text = 'Area: ' + (total_area * 100).toFixed(3) + '%';
    source.change.emit();

    function norm_cdf(value) {
        return (1.0 + erf(value / Math.sqrt(2))) / 2.0;
    }

    function erf(x) {
        // Numerical approximation of error function
        const a1 =  0.254829592;
        const a2 = -0.284496736;
        const a3 =  1.421413741;
        const a4 = -1.453152027;
        const a5 =  1.061405429;
        const p  =  0.3275911;

        const t = 1.0 / (1.0 + p * Math.abs(x));
        const y = 1 - (((((a5 * t + a4) * t) + a3) * t + a2) * t + a1) * t * Math.exp(-x * x);

        return x >= 0 ? y : -y;
    }
""")

# Sliders for left and right limits
w = 410
left_slider = Slider(start=-5, end=5, value=-5, step=0.1, title="Left Limit", width = w)
right_slider = Slider(start=-5, end=5, value=5, step=0.1, title="Right Limit", width=w)

left_slider.js_on_change('value', callback)
right_slider.js_on_change('value', callback)

callback.args["left_slider"] = left_slider
callback.args["right_slider"] = right_slider

layout = column(left_slider, right_slider, area_text, plot)

#output_file("normalplotthing.html")

# Show plot
# Define the CSS style
style = """
<style>
.output {
    display: flex;
    align-items: center;
    justify-content: center;
}
</style>
"""

# Apply the style
display(HTML(style))



show(layout)